https://www.banki.ru/banks/ratings/?source=submenu_banksratings

In [1]:
import requests  
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
from tqdm.notebook import tqdm

в этой урле (sample_url) есть дата 1 и дата 2 

надо пройти как бы скользящей штукой дата1 дата2 - дата 2 дата 3 видимо чтобы скачать для 1 показателя за весь промежуток

те тут остается найти на сайте где это все лежит и по каждой ссылке пройтись итерируясь по датам

потом как то надо эти все датасеты мерджить, возможно надо продумать чтобы сразу выкидывать ненужные колонки и оставлять только нужные, по которым будет мердж, колонки - это месяца, изменение мб потом самостоятельно посчитать, а мб и оставить

In [ ]:
sample_url = 'https://www.banki.ru/banks/ratings/export.php?SEARCH_NAME=&SEARCH_REGN=&search[type]=name&sort_param=rating&sort_order=ASC&PROPERTY_ID=10&REGION_ID=0&date1=2022-02-01&date2=2008-03-01&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0'

In [316]:
url = 'https://www.banki.ru/banks/ratings/export.php?SEARCH_NAME=&SEARCH_REGN=&search[type]=name&sort_param=rating&sort_order=ASC&PROPERTY_ID=10&REGION_ID=0&date1=2008-03-01&date2=&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0'
response = requests.get(url)

x = BeautifulSoup(response.content, 'html.parser')
bald_text = x.text
raw_text = bald_text.split('\r\n')

pokazatel_raw = raw_text[1]
_, pokazatel = pokazatel_raw.split(': ') #какой показатель парсим например: Активы нетто 

header_columns = raw_text[3].split(';') #название колонок датасета
df = pd.DataFrame(columns=header_columns)
df_rows = raw_text[4:-1]    #контент датасета

loc_num = 0
for i in df_rows:
    lst_i = i.split(';')
    df.loc[loc_num] = lst_i
    loc_num +=1  #index
    

In [317]:
url = 'https://www.banki.ru/banks/ratings/export.php?SEARCH_NAME=&SEARCH_REGN=&search[type]=name&sort_param=rating&sort_order=ASC&PROPERTY_ID=10&REGION_ID=0&date1=2008-06-01&date2=2008-05-01&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0'
response = requests.get(url)

x = BeautifulSoup(response.content, 'html.parser')
bald_text = x.text
raw_text = bald_text.split('\r\n')

pokazatel_raw = raw_text[1]
_, pokazatel = pokazatel_raw.split(': ') #какой показатель парсим например: Активы нетто 

header_columns = raw_text[3].split(';') #название колонок датасета
df1 = pd.DataFrame(columns=header_columns)
df_rows = raw_text[4:-1]    #контент датасета

loc_num = 0
for i in df_rows:
    lst_i = i.split(';')
    df1.loc[loc_num] = lst_i
    loc_num +=1  #index

In [318]:
df = df[df.columns[[2, 5, 6]]]
df1 = df1[df1.columns[[2, 5, 6]]]

In [319]:
df.merge(df1, how='left', on = 'Название банка')

,Название банка,"Март, 2008, тыс. рублей","Январь, 2022, тыс. рублей","Июнь, 2008, тыс. рублей","Май, 2008, тыс. рублей"
0,Сбербанк,"5 237 526 353,00","38 631 723 561,00","5 540 903 631,00","5 402 857 683,00"
1,ВТБ,"1 522 719 851,00","19 484 666 238,00","1 723 665 572,00","1 608 399 146,00"
2,Газпромбанк,"852 116 507,00","8 620 265 128,00","956 719 991,00","899 638 155,00"
3,Альфа-Банк,"541 253 952,00","5 726 461 718,00","591 924 266,00","589 554 941,00"
4,БМ-Банк,"499 808 435,00","609 549 508,00","587 010 399,00","561 037 630,00"
...,...,...,...,...,...
960,Банк «Континенталь»,"13 169,00","462 539,00","9 254,00","13 326,00"
961,Газтрансбанк,"11 907,00","8 401 982,00","12 438,00","14 392,00"
962,Каспий,"10 428,00",-,"9 663,00","8 887,00"
963,НКО «Платежи и Расчеты»,"6 154,00","1 243 981,00","4 227,00","4 314,00"


In [320]:
df_init = pd.DataFrame()

In [323]:
df

,Название банка,"Март, 2008, тыс. рублей","Январь, 2022, тыс. рублей"
0,Сбербанк,"5 237 526 353,00","38 631 723 561,00"
1,ВТБ,"1 522 719 851,00","19 484 666 238,00"
2,Газпромбанк,"852 116 507,00","8 620 265 128,00"
3,Альфа-Банк,"541 253 952,00","5 726 461 718,00"
4,БМ-Банк,"499 808 435,00","609 549 508,00"
...,...,...,...
952,Банк «Континенталь»,"13 169,00","462 539,00"
953,Газтрансбанк,"11 907,00","8 401 982,00"
954,Каспий,"10 428,00",-
955,НКО «Платежи и Расчеты»,"6 154,00","1 243 981,00"


In [324]:
df1

,Название банка,"Июнь, 2008, тыс. рублей","Май, 2008, тыс. рублей"
0,Сбербанк,"5 540 903 631,00","5 402 857 683,00"
1,ВТБ,"1 723 665 572,00","1 608 399 146,00"
2,Газпромбанк,"956 719 991,00","899 638 155,00"
3,Россельхозбанк,"605 084 306,00","540 303 040,00"
4,Альфа-Банк,"591 924 266,00","589 554 941,00"
...,...,...,...
959,Газтрансбанк,"12 438,00","14 392,00"
960,Каспий,"9 663,00","8 887,00"
961,Банк «Континенталь»,"9 254,00","13 326,00"
962,Нафтабанк,"4 920,00","4 638,00"


In [332]:
x1 = df.to_dict('records')

In [331]:
x2 = df1.to_dict('records')

In [330]:
l = []

In [334]:
l.append(x2)

,Название банка,"Март, 2008, тыс. рублей","Январь, 2022, тыс. рублей"
0,Сбербанк,"5 237 526 353,00","38 631 723 561,00"
1,ВТБ,"1 522 719 851,00","19 484 666 238,00"
2,Газпромбанк,"852 116 507,00","8 620 265 128,00"
3,Альфа-Банк,"541 253 952,00","5 726 461 718,00"
4,БМ-Банк,"499 808 435,00","609 549 508,00"
...,...,...,...
952,Банк «Континенталь»,"13 169,00","462 539,00"
953,Газтрансбанк,"11 907,00","8 401 982,00"
954,Каспий,"10 428,00",-
955,НКО «Платежи и Расчеты»,"6 154,00","1 243 981,00"


https://www.banki.ru/banks/ratings/export.php?SEARCH_NAME=&SEARCH_REGN=&search[type]=name&sort_param=rating&sort_order=ASC&PROPERTY_ID=10&REGION_ID=0&date1=2008-04-01&date2=&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0

https://www.banki.ru/banks/ratings/export.php?SEARCH_NAME=&SEARCH_REGN=&search[type]=name&sort_param=rating&sort_order=ASC&PROPERTY_ID=10&REGION_ID=0&date1=2008-05-01&date2=2008-03-01&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0

In [ ]:
date1 = may #дата конца
date2 = march #дата начала

In [469]:
# dataframes_list = []
# list1 = ['01','02','03','04','05','06','07','08','09', '10', '11', '12']
# indexes = np.arange(len(list1))
# for i, index in tqdm(zip(list1, indexes), total=len(list1)):
#     data2 = f"2008-{list1[index]}-01"
#     if index != 11:
#         data1 = f"2008-{list1[index+1]}-01"
#     else:
#         break
    
#     url = f'https://www.banki.ru/banks/ratings/export.php?SEARCH_NAME=&SEARCH_REGN=&search[type]=name&sort_param=rating&sort_order=ASC&PROPERTY_ID=10&REGION_ID=0&date1={data1}&date2={data2}&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0' 
    
#     #выгружаем инфу из url
#     response = requests.get(url)

#     x = BeautifulSoup(response.content, 'html.parser')
#     bald_text = x.text
#     raw_text = bald_text.split('\r\n')

#     pokazatel_raw = raw_text[1]
#     _, pokazatel = pokazatel_raw.split(': ') #какой показатель парсим например: Активы нетто 

#     header_columns = raw_text[3].split(';') #название колонок датасета
#     df = pd.DataFrame(columns=header_columns) #
#     df_rows = raw_text[4:-1]    #контент датасета

#     loc_num = 0
#     for i in df_rows:
#         lst_i = i.split(';')
#         df.loc[loc_num] = lst_i
#         loc_num +=1  #index
        
#     # преобразования таблицы
#     df = df[df.columns[[2, 5, 6]]] #оставляем только название банка и значение показателя на дату2 и дату1
#     df_i = df.to_dict('records')
    
#     dataframes_list.append(df_i)
    
#     time.sleep(np.random.randint(2, 5)) #после прохода по ссылке спим 3 сек

# #цикл завершился  - собираем единую табличку за год
# dataframes_year_list = dataframes_list[1:]
# init_table = pd.DataFrame(dataframes_year_list[0])

# for table in dataframes_list1[1:]:
#     table_i = pd.DataFrame(table)
    
#     init_table = init_table.merge(table_i, how = 'outer', on = 'Название банка')

# year_table = init_table[init_table.columns[[0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19]]]
# year_table = year_table.drop_duplicates(subset='Название банка')
# year_table.columns = year_table.columns.str.replace('_x', '') #финальная табличка за год

In [470]:
# dates2 = ['01','03','05','07','09','11']
# dates1 = ['02','04','06','08','10','12']
# for d2, d1 in tqdm(zip(dates2, dates1), total=len(dates2)):
#     data2 = f"2009-{d2}-01"
#     data1 = f"2009-{d1}-01"
    
#     url = f'https://www.banki.ru/banks/ratings/export.php?SEARCH_NAME=&SEARCH_REGN=&search[type]=name&sort_param=rating&sort_order=ASC&PROPERTY_ID=10&REGION_ID=0&date1={data1}&date2={data2}&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0' 
#     print(url)



In [7]:
# dataframes_list = []
# year = '2015'

# dates2 = ['01','03','05','07','09','11']
# dates1 = ['02','04','06','08','10','12']
# for d2, d1 in tqdm(zip(dates2, dates1), total=len(dates2)):
#     data2 = f"{year}-{d2}-01"
#     data1 = f"{year}-{d1}-01"
    
#     url = f'https://www.banki.ru/banks/ratings/export.php?SEARCH_NAME=&SEARCH_REGN=&search[type]=name&sort_param=rating&sort_order=ASC&PROPERTY_ID=10&REGION_ID=0&date1={data1}&date2={data2}&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0' 
#     #выгружаем инфу из url
#     response = requests.get(url)

#     x = BeautifulSoup(response.content, 'html.parser')
#     bald_text = x.text
#     raw_text = bald_text.split('\r\n')

#     pokazatel_raw = raw_text[1]
#     _, pokazatel = pokazatel_raw.split(': ') #какой показатель парсим например: Активы нетто 

#     header_columns = raw_text[3].split(';') #название колонок датасета
#     df = pd.DataFrame(columns=header_columns) #
#     df_rows = raw_text[4:-1]    #контент датасета

#     loc_num = 0
#     for i in df_rows:
#         lst_i = i.split(';')
#         df.loc[loc_num] = lst_i
#         loc_num +=1  #index
        
#     # преобразования таблицы
#     df = df[df.columns[[2, 5, 6]]] #оставляем только название банка и значение показателя на дату2 и дату1
#     df_i = df.to_dict('records')
    
#     dataframes_list.append(df_i)
    
#     time.sleep(np.random.randint(2, 5)) #после прохода по ссылке спим 3 сек
    
# #цикл завершился  - собираем единую табличку за год
# init_table = pd.DataFrame(dataframes_list[0])

# for table in dataframes_list[1:]: #проходимся по всем оставшимся кроме 1 таблички, тк к ней будем все клеить
#     table_i = pd.DataFrame(table)
#     init_table = init_table.merge(table_i, how = 'outer', on = 'Название банка')

# #обрабатываем таблицу
# init_table.columns = init_table.columns.str.replace(', тыс. рублей', '')
# init_table.columns = init_table.columns.str.replace(', ', '_')
# year_table = init_table[['Название банка', #получаем финальную таблицу за год
#             f'Январь_{year}',   
#             f'Февраль_{year}',
#             f'Март_{year}',
#             f'Апрель_{year}',
#             f'Май_{year}',
#             f'Июнь_{year}',
#             f'Июль_{year}',
#             f'Август_{year}',
#             f'Сентябрь_{year}',
#             f'Октябрь_{year}',
#             f'Ноябрь_{year}',
#             f'Декабрь_{year}']]

In [476]:
# year_table_2009 = year_table[year_table.columns]

In [480]:
# year_table_2015 = year_table[year_table.columns]

In [6]:
# year_table_2009.merge(year_table_2015, how = 'outer', on = 'Название банка')

In [5]:
# years_list = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

# for year in years_list:
   

In [3]:
# init_table = pd.DataFrame(year_table_2009['Название банка'])

In [4]:
# init_table.merge(year_table_2015, how = 'outer', on = 'Название банка').merge(year_table_2009, how = 'outer', on = 'Название банка')

In [506]:
# initial_table = pd.DataFrame(columns = ['Название банка'])
# initial_table['Название банка'] = ['Сбербанк', 'ВТБ', 'Газпромбанк']

In [ ]:
=======

In [2]:
# years_list = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

years_list = ['2009', '2014', '2021']
total_pokazatel_table = pd.DataFrame(columns = ['Название банка'])
total_pokazatel_table['Название банка'] = ['Сбербанк', 'ВТБ', 'Газпромбанк']

for year in tqdm(years_list):

    dataframes_list = []

    dates2 = ['01','03','05','07','09','11']
    dates1 = ['02','04','06','08','10','12']
    for d2, d1 in tqdm(zip(dates2, dates1), total=len(dates2)):
        data2 = f"{year}-{d2}-01"
        data1 = f"{year}-{d1}-01"

        url = f'https://www.banki.ru/banks/ratings/export.php?SEARCH_NAME=&SEARCH_REGN=&search[type]=name&sort_param=rating&sort_order=ASC&PROPERTY_ID=10&REGION_ID=0&date1={data1}&date2={data2}&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0' 
        #выгружаем инфу из url
        response = requests.get(url)

        x = BeautifulSoup(response.content, 'html.parser')
        bald_text = x.text
        raw_text = bald_text.split('\r\n')

        pokazatel_raw = raw_text[1]
        _, pokazatel = pokazatel_raw.split(': ') #какой показатель парсим например: Активы нетто 

        header_columns = raw_text[3].split(';') #название колонок датасета
        df = pd.DataFrame(columns=header_columns) #
        df_rows = raw_text[4:-1]    #контент датасета

        loc_num = 0
        for i in df_rows:
            lst_i = i.split(';')
            df.loc[loc_num] = lst_i
            loc_num +=1  #index

        # преобразования таблицы
        df = df[df.columns[[2, 5, 6]]] #оставляем только название банка и значение показателя на дату2 и дату1
        df_i = df.to_dict('records')

        dataframes_list.append(df_i)

        time.sleep(np.random.randint(3, 6)) #после прохода по ссылке спим 3 сек

    #цикл завершился  - собираем единую табличку за год
    init_table = pd.DataFrame(dataframes_list[0])

    for table in dataframes_list[1:]: #проходимся по всем оставшимся кроме 1 таблички, тк к ней будем все клеить
        table_i = pd.DataFrame(table)
        init_table = init_table.merge(table_i, how = 'outer', on = 'Название банка')

    #обрабатываем таблицу
    init_table.columns = init_table.columns.str.replace(', тыс. рублей', '')
    init_table.columns = init_table.columns.str.replace(', ', '_')
    year_table = init_table[['Название банка', #получаем финальную таблицу за год
                f'Январь_{year}',   
                f'Февраль_{year}',
                f'Март_{year}',
                f'Апрель_{year}',
                f'Май_{year}',
                f'Июнь_{year}',
                f'Июль_{year}',
                f'Август_{year}',
                f'Сентябрь_{year}',
                f'Октябрь_{year}',
                f'Ноябрь_{year}',
                f'Декабрь_{year}']]

    total_pokazatel_table = total_pokazatel_table.merge(year_table, how = 'outer', on = 'Название банка')
    
    total_pokazatel_table = total_pokazatel_table.drop_duplicates(subset='Название банка', keep = 'last')
    #разобраться почему вылезают какие то странные дубликаты
    
    total_pokazatel_table = total_pokazatel_table.fillna(-1)
    total_pokazatel_table[total_pokazatel_table.columns[1:]] = total_pokazatel_table[total_pokazatel_table.columns[1:]].applymap(
    lambda x: str(x).replace(",00", '')) #убираем ,00

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

C:\Users\Masha\AppData\Local\Temp\ipykernel_40248\528369266.py:54: FutureWarning: The default value of regex will change from True to False in a future version.
  init_table.columns = init_table.columns.str.replace(', тыс. рублей', '')


  0%|          | 0/6 [00:00<?, ?it/s]

C:\Users\Masha\AppData\Local\Temp\ipykernel_40248\528369266.py:54: FutureWarning: The default value of regex will change from True to False in a future version.
  init_table.columns = init_table.columns.str.replace(', тыс. рублей', '')


  0%|          | 0/6 [00:00<?, ?it/s]

C:\Users\Masha\AppData\Local\Temp\ipykernel_40248\528369266.py:54: FutureWarning: The default value of regex will change from True to False in a future version.
  init_table.columns = init_table.columns.str.replace(', тыс. рублей', '')


In [8]:
total_pokazatel_table.to_excel('total_assets.xlsx')